Age and Gender Detection is the task of **Computer Vision** so we will be using **OpenCV** library. However before getting started with this task, we will first go through the concept and how to deal with the problem of age and gender detection.

Understanding the concept is important so that in future we can easily perform related task 

### Introduction to Age and Gender Detection

The task of detecting age and gender, however, is an inherently difficult problem, more than many other computer vision tasks. The main reason for this difficulty, gap lies in the data required to train these types of systems.

While general object detection tasks can often have access to hundreds of thousands or even millions of images for training, datasets with age and/or gender labels are considerably smaller, usually in the thousands or, at best, tens of thousands. The reason is that, to have tags for such images, we need to access the personal information of the subjects in the images. Namely, we would need their date of birth and gender, and in particular date of birth is infrequently published information. Therefore, we have to settle for the nature of this problem that we are addressing and adapt network architectures and algorithmic approaches to deal with these limitations.

The areas of classification by age and sex have been studied for decades. Various approaches have been taken over the years to tackle this problem, with varying levels of success.

We will present the problem of gender detection as a classification problem and the age detection problem as a regression problem. However, estimating age accurately using regression is difficult. Even humans cannot accurately predict an age by looking at a person. However, we do know if they are in their `30s` or `40s`. This is also what we are going to follow.

Now let’s get started with the task of Age and Gender detection. We will first start with writing the code for detecting faces because without face detection we will not be able to move further with the task of age and gender prediction.

We can download the necessary OpenCV pre-trained models that we will need in the task of age and gender detection from [here](https://drive.google.com/file/d/1yy_poZSFAPKi0y2e2yj9XDe1N8xXYuKB/view) or [here](https://github.com/opencv/opencv/tree/master/data/haarcascades). 

In [19]:
# pip install opencv-python
import cv2
import math
import argparse

In [20]:
def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn=frame.copy()
    frameHeight=frameOpencvDnn.shape[0]
    frameWidth=frameOpencvDnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections=net.forward()
    faceBoxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn,faceBoxes

Next step is to predict gender and age in the image. We will load the gender and ageing network into memory and transmit the detected face across the network for the gender and age detection task.

In [23]:
parser=argparse.ArgumentParser()
parser.add_argument('--image')

args=parser.parse_args()

faceProto="cv/opencv_face_detector.pbtxt"
faceModel="cv/opencv_face_detector_uint8.pb"
ageProto="cv/age_deploy.prototxt"
ageModel="cv/age_net.caffemodel"
genderProto="cv/gender_deploy.prototxt"
genderModel="cv/gender_net.caffemodel"

MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746)
ageList=['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList=['Male','Female']

faceNet=cv2.dnn.readNet(faceModel,faceProto)
ageNet=cv2.dnn.readNet(ageModel,ageProto)
genderNet=cv2.dnn.readNet(genderModel,genderProto)

video=cv2.VideoCapture(args.image if args.image else 0)
padding=20
while cv2.waitKey(1)<0:
    hasFrame,frame=video.read()
    if not hasFrame:
        cv2.waitKey()
        break

    resultImg,faceBoxes=highlightFace(faceNet,frame)
    if not faceBoxes:
        print("No face detected")

    for faceBox in faceBoxes:
        face=frame[max(0,faceBox[1]-padding):
                   min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding)
                   :min(faceBox[2]+padding, frame.shape[1]-1)]
        
        # Code for Gender Detection:
        blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds=genderNet.forward()
        gender=genderList[genderPreds[0].argmax()]
        print("Gender Output : {}".format(genderPreds))
        print(f'Gender: {gender}')

        # Code for Age Detection:
        ageNet.setInput(blob)
        agePreds=ageNet.forward()
        age=ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print(f'Age: {age[1:-1]} years')
        
        #  last code we need to write is to display the output:
        label = f'{gender}, {age}'
        cv2.putText(resultImg, label, (faceBox[0], faceBox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                    (0,255,255), 2, cv2.LINE_AA)
        cv2.imshow("Detecting age and gender", resultImg)

usage: ipykernel_launcher.py [-h] [--image IMAGE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Waqas.Ali\AppData\Roaming\jupyter\runtime\kernel-738ac2ab-2a39-4a54-8934-282b62f54ab1.json


SystemExit: 2

We cannot experiment with this easily in ipython as the ipython command line will be used per default in `parse_args`. Try this with a normal `*.py` file and the python executable to launch that file. 

run following on command line

`python gender_age.py`